In [3]:
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
#from keras.utils import plot_model

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import csv
import cv2
import os
from numpy import genfromtxt
import requests
import re


In [5]:
#letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
#path = "./ASL_CSV/train/"
#df = pd.read_csv("ASL_CSV/train/A/A1.csv", sep = ',')

letter = "A"
num = 2
df = pd.read_csv('./ASL_CSV/train/'+ letter + '/' + letter + str(num) + '.csv', sep=',', header=None)

array = df.to_numpy()
result = array.flatten()
string = ""
for x in result:
    replaced = re.sub("[^0-9 ]+", "", x)
    replaced = re.sub(' +', ' ', replaced).strip()
    string += replaced
string = string.split(" ")
string.insert(0, "A")
df2 = pd.DataFrame(columns=np.arange(39802))
df2.rename(columns={0:"Labels"}, inplace = True)
df2.loc[len(df2)] = string
print(len(string)) #Bruh why is this list only 39802 = should be 20X20
print(df2.shape[1])
print(df2)


    




39802
39802
  Labels   1   2   3   4   5   6   7   8   9  ... 39792 39793 39794 39795  \
0      A  33  30  31  30  29  30  29  28  28  ...    28    28    26    29   

  39796 39797 39798 39799 39800 39801  
0    29    29    29    29    30    28  

[1 rows x 39802 columns]
